In [1]:
#| default_exp ECMs.episodic

In [2]:
#| export
import numpy as np
from projective_simulation.ECMs.template_ECM import ECM
from projective_simulation.utils import _softmax

class Episodic_Memory(ECM):
    def __init__(self,
                 num_actions: int,
                 capacity: int = 10,
                 softmax: float = 0.7,
                 t: int = 0,
                 focus: float = 1.,
                 error_tolerance: float = 0.01,
                 min_expectation: float = 0.01
                ):
        #initialize constants
        self.num_actions = num_actions
        self.softmax = 0.7 #temperature for softmax function
        self.focus = focus
        self.error_tolerance = error_tolerance
        self.min_expectation
        
        #initialize variables
        self.t = t

        #initialize ECM states
        self.hmatrix = np.zeros((num_actions, capacity)) #weights connecting percept nodes to trace nodes
        self.capacity = capacity
        self.trace_encoder = np.zeros((self.num_actions, self.capacity)) #boolean matrix indicating whether a percept node was encoded in a trace
        self.action_encoder = np.ones((self.num_actions, self.capacity)) #boolean matrix indicated whether a percept node belongs to an action
        self.expectations = np.zeros(self.num_actions) + self.min_expectation
        self.beliefs = np.zeros(self.capacity)
        self.valences = np.zeros(self.capacity)
        

    def deliberate(self, percept):
        self.add_percept(percept)
        self.encode_trace(percept)

    def add_percept(self, percept):
        if len(percept) > np.shape(self.hmatrix)[0]: #if percept is longer than the first dimension of the hmatrix it means there is something new in the observation (handled by preprocessor)
            i = np.shape(self.hmatrix)[0] #get index for new elements in ECM
            new_elements = percept[i:len(percept)]    
            self.hmatrix = np.append(self.hmatrix, np.zeros((len(new_elements), self.capacity)), axis = 0) #add baseline weights to connections between new percept nodes and all traces
            self.trace_encoder = np.append(self.trace_encoder, np.zeros((len(new_elements), self.capacity)), axis = 0) #new percept nodes have no existing connections to trace nodes
            self.action_encoded = np.append(self.action_encoder, np.zeros((len(new_elements), self.capacity)), axis = 0) #this ECM does not support new actions, so new percept nodes are sensory by default

    def encode_trace(self, percept):
        self.trace_encoder[0:,self.t] = percept

In [3]:
test = Episodic_Memory(num_actions = 2)
test.trace_encoder



array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [4]:
test.deliberate([1,0,1,1])
test.trace_encoder

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])